# Let's do some setup first

Let's test the API first and the setup

Let's measure how much time it requires to get a prediction from clarifai

On a un problème de scalabilité : 1 seconde pour une classification


In [2]:
from clarifai import rest
from clarifai.rest import ClarifaiApp
import numpy as np
import time

app = ClarifaiApp("MkJ4CqC10UXCw8lFZrp04JIntyLidwU375tFSusY", "-UgMOcuirnzYH4Oei2bVTI4ekXbVPw5LvcEpfkIb")
# model = app.models.get('testing')
model = app.models.get("general-v1.3")
t = model.get_info(verbose=True)

print (model.predict_by_url(url='http://www.journaldugeek.com/files/2016/07/Jeff-Bezos.jpg'))

# I measure the time required for one request

start_time = time.time()
r = model.predict_by_url(url='https://previews.123rf.com/images/iwansntu/iwansntu1311/iwansntu131100004/24428202-Wide-view-of-old-railway-track-around-Rifle-Range-Road-Bukit-Timah-Singapore-Stock-Photo.jpg')
print(r)
print("--- %s seconds ---" % (time.time() - start_time))



Hey! Clarifai Python Client v2.0.29 upgrade available.


{'status': {'description': 'Ok', 'code': 10000}, 'outputs': [{'input': {'data': {'image': {'url': 'http://www.journaldugeek.com/files/2016/07/Jeff-Bezos.jpg'}}, 'id': 'fee6a3a6dbb042b7b6eecb82468dfd70'}, 'created_at': '2017-07-04T20:21:44.822024923Z', 'model': {'model_version': {'status': {'description': 'Model trained successfully', 'code': 21100}, 'created_at': '2016-07-13T01:19:12.147644Z', 'id': 'aa9ca48295b37401f8af92ad1af0d91d'}, 'app_id': 'main', 'created_at': '2016-03-09T17:11:39.608845Z', 'name': 'general-v1.3', 'id': 'aaa03c23b3724a16a56b629203edc62c', 'output_info': {'type_ext': 'concept', 'message': 'Show output_info with: GET /models/{model_id}/output_info', 'type': 'concept'}}, 'status': {'description': 'Ok', 'code': 10000}, 'id': 'b4c1e544e4e34597bb17bc9571ac455c', 'data': {'concepts': [{'value': 0.9912883, 'name': 'man', 'app_id': 'main', 'id': 'ai_dxSG2s86'}, {'value': 0.9802798, 'name': 'people', 'app_id': 'main', 'id': 'ai_l8TKp2h5'}, {'value': 0.97664106, 'name': 'p

# List of available Concepts

Let's try to get the list of available concepts in the Clarifai generak engine

In [4]:
concepts = t['model']['output_info']['data']['concepts']

# print only the names of the objects
names_list = list( map(lambda x:x['name'] , concepts) )
names_list.sort()


D = len(names_list)

# Create a feature map

In [3]:
import os

fmap = {}
for i in range(len(names_list)):
    fmap[names_list[i]] = i


def get_classes(uniquePred):
    return list(map(lambda x: ({'name': x['name'], 'value' : x['value']}), uniquePred))

def clean_features(pred) :
    # get the predictins only
    cpred =  list(map(lambda x : x['outputs'][0]['data']['concepts'], pred))
    # get the predictions name and value only
    cpred = list(map (lambda x : get_classes(x), cpred))
    return cpred

def pred_from_file(fname, model):
    content = list()
    with open(fname) as f:
        urlList = f.readlines()
        # remove whitespace characters like `\n` at the end of each line
        urlList = [x.strip() for x in urlList]
        urlList = list(filter(lambda x: x != '', urlList))   # remove blanks
        url_class = list(map(lambda x: x.split(','), urlList))
        pred = list(map(lambda urlc : model.predict_by_url(urlc[0]),url_class))
        y = list(map(lambda x: int(x[1]), url_class))
    f.close()
    return pred, y

def pred_from_folder(folname, model)

def create_feature_map(fname, model):
    pred, y = pred_from_file(fname, model) # create predictions list
    c_features = clean_features(pred) # clean the predictions to keep the features only
    X = featureMap(c_features, fmap) # Now construct a feature map
    return X, y

def featureMap(c_features, fmap):
    N, D = len(c_features), len(fmap)
    X = np.zeros((N,D))
    for i in range(N):
        for f in c_features[i]:
            pos = fmap[f['name']]
            X[i, pos] = f['value']
    return X

X, y = create_feature_map('untitled.txt', model)
        

# Train a classifier

We train a small classifier on the files' dataset and see the accuracy we could achieve

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# split to a training and validation 
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)

clf = RandomForestClassifier(n_estimators=10)
clf = clf.fit(X_train, y_train)
